In [53]:
print('hello world')

hello world


In [56]:
import sys
sys.path.append('/home/s94zalek/shape_matching')

from datasets_code import build_dataloader, build_dataset
from utils.options import parse_options
from train import create_train_val_dataloader
import torch
import numpy as np

import os
os.chdir('/home/s94zalek/shape_matching')

# print current working directory
print(os.getcwd())

/home/s94zalek/shape_matching


In [57]:
import os

# root_path = osp.abspath(osp.join(__file__, osp.pardir))
root_path = '/home/s94zalek/shape_matching'

opt = parse_options(root_path, is_train=False, use_argparse=False,
                    opt_path = 'options/train/faust.yaml')

opt['root_path'] = root_path
opt['dist'] = False

opt['datasets']['train_dataset']['return_corr'] = True
opt['datasets']['train_dataset']['return_dist'] = False
opt['datasets']['test_dataset']['return_dist'] = False

# create train and validation dataloaders
result = create_train_val_dataloader(opt)
train_loader, train_sampler, val_loader, total_epochs, total_iters = result

test_set = build_dataset(opt['datasets']['test_dataset'])
test_loader = build_dataloader(
test_set, opt['datasets']['test_dataset'], phase='val', num_gpu=opt['num_gpu'], dist=opt['dist'], sampler=None, seed=opt['manual_seed'])


Backend DataParallel.
Path already exists. Rename it to /home/s94zalek/shape_matching/results/faust_archived_20240510_153944


Calculating functional maps: 100%|██████████████████████████████████████████████████████████████████████████████████| 400/400 [00:03<00:00, 102.84it/s]


In [ ]:
import trimesh

scene = trimesh.Scene()

In [ ]:
# !python preprocess.py --data_root data/TOPKIDS/ --no_normalize --no_dist --n_eig 200

In [ ]:
# add random sampler to test_loader
test_loader.dataset.sampler = 'random'

In [ ]:
from tqdm import tqdm

# data_x = test_set[15]['first']
# data_y = test_set[15]['second']

# get 15th element from test dataloader
for i, data in enumerate(tqdm(test_loader)):
    if i == 10:
        data_x = data['first']
        data_y = data['second']
        break

list(data_x.keys())

In [ ]:
scene.geometry.clear()

mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())
scene.add_geometry(mesh1)

mesh2 = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())
scene.add_geometry(mesh2)

scene.show()

In [ ]:
print('data_x', data_x['name'], 'data_y', data_y['name'])

Pxy = -torch.ones(data_x['verts'].shape[1], dtype=torch.int64)
print('Pxy.shape', Pxy.shape)
Pxy[data_x['corr']] = data_y['corr']

Pyx = -torch.ones(data_y['verts'].shape[1], dtype=torch.int64)
print('Pyx.shape', Pyx.shape)
Pyx[data_y['corr']] = data_x['corr']

# check 
range_1_len = torch.arange(data_x['verts'].shape[1])
print('Pxy', Pxy[:20])
print('Pyx', Pyx[:20])
print(torch.cat((range_1_len.view(-1, 1), Pxy.view(-1, 1)), dim=1)[:20])

In [ ]:
# print number of unique elements in Pxy and count of each element, sorted by count
unique, counts = Pxy.unique(return_counts=True)

# print the values sorted by count
print('unique', unique[torch.argsort(counts, descending=True)])
print('counts', counts[torch.argsort(counts, descending=True)])

# Are raw meshes aligned?

In [ ]:
import trimesh
import numpy as np

# clear the scene
scene.geometry.clear()

# cmap = trimesh.visual.color.interpolate(np.linspace(0, 1, len(data_y['verts'][0])), 'jet')

# make a color map based on y coordinate
cmap = trimesh.visual.color.interpolate(data_x['verts'][0][:, 1], 'jet')


# add the first mesh
mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())
mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)]
scene.add_geometry(mesh1)

mesh2 = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())
mesh2.visual.vertex_colors = cmap[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

# print('distance between vertices in dataset and vertices in mesh object:',
#       (data_x['verts'][0].numpy() - mesh1.vertices).sum()
#       )

# display the scene
scene.show('notebook')

# After applying correspondences

In [ ]:
def index_with_P(tensor_to_index, P):
    # assert tensor_to_index.shape[0] == P.shape[0], f'tensor_to_index.shape {tensor_to_index.shape} != P.shape {P.shape}'
    
    indexed_tensor = tensor_to_index[P]
    
    # replace values where P = -1 with zeros
    if type(indexed_tensor) == torch.Tensor:
        backend = torch 
    elif type(indexed_tensor) == np.ndarray:
        backend = np
    else:
        raise ValueError(f'Unknown type of indexed_tensor {type(indexed_tensor)}')
    
    indexed_tensor[P == -1] = backend.zeros_like(indexed_tensor[P == -1])
    return indexed_tensor
    

# X -> Y


In [ ]:
#########################################
# X -> Y
#########################################

scene.geometry.clear()

cmap = trimesh.visual.color.interpolate(data_x['verts'][0][:, 1], 'jet')


# add the first mesh
mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())
mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)]
scene.add_geometry(mesh1)



mesh2 = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())

# cmap2 = cmap[Pyx][:len(mesh2.vertices)]
cmap2 = index_with_P(cmap, Pyx)[:len(mesh2.vertices)]
print(cmap2.shape, len(mesh2.vertices))

mesh2.visual.vertex_colors = cmap2
scene.add_geometry(mesh2)

scene.show()

# Y -> X

In [ ]:
#########################################
# Y -> X
#########################################

scene.geometry.clear()

cmap2 = trimesh.visual.color.interpolate(data_y['verts'][0][:, 1], 'hsv')


# add the second mesh
mesh2 = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())
mesh2.visual.vertex_colors = cmap2[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())
cmap = index_with_P(cmap2, Pxy)
mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)]
scene.add_geometry(mesh1)

scene.show()

# Are eigenfunctions orthogonal?

In [ ]:
phi_T_phi = data_x['evecs_trans'][0] @ data_x['evecs'][0]
phi_phi_T = data_x['evecs'][0] @ data_x['evecs_trans'][0]

print('Shape X')
print('phi.shape', data_x['evecs'][0].shape)
print('phi_T_phi.shape:', phi_T_phi.shape, ', dist(phi_T @ phi, I):', (phi_T_phi - torch.eye(phi_T_phi.shape[0])).abs().sum())
print('phi_phi_T.shape:', phi_phi_T.shape, ', dist(phi @ phi_T, I):', (phi_phi_T - torch.eye(phi_phi_T.shape[0])).abs().sum())


# do the same for y
phi_T_phi = data_y['evecs_trans'][0] @ data_y['evecs'][0]
phi_phi_T = data_y['evecs'][0] @ data_y['evecs_trans'][0]

print()
print('Shape Y')
print('phi.shape', data_y['evecs'][0].shape)
print('phi_T_phi.shape:', phi_T_phi.shape, ', dist(phi_T @ phi, I):', (phi_T_phi - torch.eye(phi_T_phi.shape[0])).abs().sum())
print('phi_phi_T.shape:', phi_phi_T.shape, ', dist(phi @ phi_T, I):', (phi_phi_T - torch.eye(phi_phi_T.shape[0])).abs().sum())


# Calculate the functional map

In [ ]:
phi_x = data_x['evecs'][0]
phi_x_T = data_x['evecs_trans'][0]
phi_y = data_y['evecs'][0]
phi_y_T = data_y['evecs_trans'][0]

Cxy_indexing_zeros = phi_y_T @ index_with_P(phi_x, Pyx)
Cyx_indexing_zeros = phi_x_T @ index_with_P(phi_y, Pxy)

# print('index_with_P(phi_x, Pyx).shape', index_with_P(phi_x, Pyx).shape)
# print('index_with_P(phi_x, Pyx)', index_with_P(phi_x, Pyx)[:10])

# print('Cxy.shape', Cxy.shape)
# print('Cxy', Cxy[:10])

# print('Cyx.shape', Cyx.shape)
# print('Cyx', Cyx[:10])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Cxy
Cxy_plot = ax[0].imshow(Cxy_indexing_zeros.cpu().numpy(), cmap='bwr', vmin=-1, vmax=1)
# ax[0].axis('off')
ax[0].set_title('Cxy_indexing_zeros')
cbar = plt.colorbar(Cxy_plot)
cbar.set_label('Cxy')

# Cyx
Cyx_plot = ax[1].imshow(Cyx_indexing_zeros.cpu().numpy(), cmap='bwr', vmin=-1, vmax=1)
# ax[1].axis('off')
ax[1].set_title('Cyx_indexing_zeros')
cbar = plt.colorbar(Cyx_plot)
cbar.set_label('Cyx')

plt.show()

# Check for orthogonality

In [ ]:
print('dist(Cyx @ Cxy, I)', ((Cyx_indexing_zeros @ Cxy_indexing_zeros) - torch.eye(Cxy_indexing_zeros.shape[0])).abs().sum())
print('dist(Cxy @ Cyx, I)', ((Cxy_indexing_zeros @ Cyx_indexing_zeros) - torch.eye(Cxy_indexing_zeros.shape[0])).abs().sum())
print('dist(Cxy @ Cxy.T, I)', ((Cxy_indexing_zeros @ Cxy_indexing_zeros.T) - torch.eye(Cxy_indexing_zeros.shape[0])).abs().sum())
print('dist(Cyx @ Cyx.T, I)', ((Cyx_indexing_zeros @ Cyx_indexing_zeros.T) - torch.eye(Cxy_indexing_zeros.shape[0])).abs().sum())

# Another method

In [ ]:
print("data_y['corr']", data_y['corr'].shape)
print("phi_y_T", phi_y_T.shape)
print("phi_y_T[:, data_y['corr'].squeeze(0)]", phi_y_T[:, data_y['corr'].squeeze(0)].shape)
print(torch.all(phi_y_T.transpose(0, 1)[data_y['corr']].transpose(1, 2) == phi_y_T[:, data_y['corr'].squeeze(0)].unsqueeze(0)))

In [ ]:
phi_x.shape

In [ ]:
phi_y[data_y['corr']].shape, phi_x.shape

In [ ]:
C_gt_xy_pinv = (torch.pinverse(phi_y[data_y['corr']]) @ phi_x[data_x['corr']])[0]
C_gt_yx_pinv = (torch.pinverse(phi_x[data_x['corr']]) @ phi_y[data_y['corr']])[0]

C_gt_xy_lstsq = torch.linalg.lstsq(phi_y[data_y['corr']], phi_x[data_x['corr']]).solution[0]
C_gt_yx_lstsq = torch.linalg.lstsq(phi_x[data_x['corr']], phi_y[data_y['corr']]).solution[0]

C_gt_xy_index = (phi_y_T.transpose(0, 1)[data_y['corr']].transpose(1, 2) @ phi_x[data_x['corr']])[0]
C_gt_yx_index = (phi_x_T.transpose(0, 1)[data_x['corr']].transpose(1, 2) @ phi_y[data_y['corr']])[0]

print('XY: pseudoinverse - indexing', (C_gt_xy_pinv - C_gt_xy_index).sum())
print('YX: pseudoinverse - indexing', (C_gt_yx_pinv - C_gt_yx_index).sum())

print('XY: pseudoinverse - lstsq', (C_gt_xy_pinv - C_gt_xy_lstsq).sum())
print('YX: pseudoinverse - lstsq', (C_gt_yx_pinv - C_gt_yx_lstsq).sum())

C_gt_xy = C_gt_xy_lstsq
C_gt_yx = C_gt_yx_lstsq

In [ ]:
# C_gt_xy = (phi_y[data_y['corr']].transpose(2, 1) @ phi_x[data_x['corr']])[0]
# C_gt_yx = (phi_x[data_x['corr']].transpose(2, 1) @ phi_y[data_y['corr']])[0]



# calculate distance to Cxy and Cyx
print('dist(Cxy, C_gt_xy):', (Cxy_indexing_zeros - C_gt_xy).abs().mean())
print('dist(Cyx, C_gt_yx):', (Cyx_indexing_zeros - C_gt_yx).abs().mean())

print('Cxy.shape', C_gt_xy.shape)

# check orthogonality for C_gt_xy and C_gt_yx
val = (C_gt_xy @ C_gt_xy.T - torch.eye(C_gt_xy.shape[0])).abs()
print(f'AbsErr(Cxy @ Cxy_T, I), sum: {val.sum().item():.4f}, mean: {val.mean().item():.4f}')

val = (C_gt_yx @ C_gt_yx.T - torch.eye(C_gt_yx.shape[0])).abs()
print(f'AbsErr(Cyx @ Cyx_T, I), sum: {val.sum().item():.4f}, mean: {val.mean().item():.4f}')

val = (C_gt_xy @ C_gt_yx - torch.eye(C_gt_xy.shape[0])).abs()
print(f'AbsErr(Cxy @ Cyx, I), sum: {val.sum().item():.4f}, mean: {val.mean().item():.4f}')

val = (C_gt_yx @ C_gt_xy - torch.eye(C_gt_yx.shape[0])).abs()
print(f'AbsErr(Cyx @ Cxy, I), sum: {val.sum().item():.4f}, mean: {val.mean().item():.4f}')

# plot them
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# C_gt_xy
C_gt_xy_plot = ax[0].imshow(C_gt_xy.cpu().numpy(), cmap='bwr', vmin=-1, vmax=1)
plt.colorbar(C_gt_xy_plot)
# ax[0].axis('off')
# ax[0].set_title('C_xy pinv')
ax[0].set_title('C_xy')

# C_gt_yx
C_gt_yx_plot = ax[1].imshow(C_gt_yx.cpu().numpy(), cmap='bwr', vmin=-1, vmax=1)
plt.colorbar(C_gt_yx_plot)
# ax[1].axis('off')
# ax[1].set_title('C_yx pinv')
ax[1].set_title('C_yx')

plt.show()

In [ ]:
print("torch.pinverse(phi_y[data_y['corr']]).sum()", torch.pinverse(phi_y[data_y['corr']]).sum())
print("phi_x[data_x['corr']].sum()", phi_x[data_x['corr']].sum())

print("torch.pinverse(phi_x[data_x['corr']]).sum()", torch.pinverse(phi_x[data_x['corr']]).sum())
print("phi_y[data_y['corr']].sum()", phi_y[data_y['corr']].sum())


In [ ]:
# use the second method
Cxy = C_gt_xy_lstsq
Cyx = C_gt_yx_lstsq

# Get the P2P map back

In [ ]:
Pyx_after = phi_y @ Cxy @ phi_x_T
Pxy_after = phi_x @ Cyx @ phi_y_T

In [ ]:
#########################################
# X -> Y
#########################################

scene.geometry.clear()

cmap = trimesh.visual.color.interpolate(data_x['verts'][0][:, 1], 'jet')

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())
mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)]
scene.add_geometry(mesh1)


mesh2 = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())

print(cmap.shape)

cmap2 = Pyx_after @ (cmap.astype(np.float32) / 255)

cmap2 = (torch.abs(cmap2).numpy() * 255).clip(0, 255).astype(np.uint8)
# cmap2[:, 3] = 255

# cmap2 = index_with_P(cmap, Pyx)[:len(mesh2.vertices)]
# print(cmap2.shape, len(mesh2.vertices))

mesh2.visual.vertex_colors = cmap2[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

scene.show()

In [ ]:
#########################################
# Y -> X
#########################################

scene.geometry.clear()

cmap2 = trimesh.visual.color.interpolate(data_y['verts'][0][:, 1], 'hsv')


# add the second mesh
mesh2 = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())
mesh2.visual.vertex_colors = cmap2[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())

cmap = Pxy_after @ (cmap2.astype(np.float32) / 255)
cmap = (torch.abs(cmap).numpy() * 255).clip(0, 255).astype(np.uint8)
cmap2[:, 3] = 255

mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)]

scene.add_geometry(mesh1)

scene.show()

# Hard correspondence matrix

In [ ]:
from utils.fmap_util import nn_query, fmap2pointmap

p2p_yx = fmap2pointmap(Cxy, phi_x, phi_y)
p2p_xy = fmap2pointmap(Cyx, phi_y, phi_x)

In [ ]:
#########################################
# X -> Y
#########################################

scene.geometry.clear()

cmap = trimesh.visual.color.interpolate(data_x['verts'][0][:, 1], 'jet')

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())
mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)]
scene.add_geometry(mesh1)


mesh2 = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())

print(cmap.shape)

# cmap2 = Pyx_after @ (cmap.astype(np.float32) / 255)

# cmap2 = (torch.abs(cmap2).numpy() * 255).clip(0, 255).astype(np.uint8)
# cmap2[:, 3] = 255

cmap2 = index_with_P(cmap, p2p_yx)[:len(mesh2.vertices)]
print(cmap2.shape, len(mesh2.vertices))

mesh2.visual.vertex_colors = cmap2[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

scene.show()

In [ ]:
#########################################
# Y -> X
#########################################

scene.geometry.clear()

cmap2 = trimesh.visual.color.interpolate(data_y['verts'][0][:, 1], 'hsv')


# add the second mesh
mesh2 = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())
mesh2.visual.vertex_colors = cmap2[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())

# cmap = Pxy_after @ (cmap2.astype(np.float32) / 255)
# cmap = (torch.abs(cmap).numpy() * 255).clip(0, 255).astype(np.uint8)
# cmap2[:, 3] = 255

cmap = index_with_P(cmap2, p2p_xy)[:len(mesh1.vertices)]

mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)]

scene.add_geometry(mesh1)

scene.show()

# Texture transfer

In [ ]:
import utils.texture_util as texture_util

uv1 = texture_util.generate_tex_coords(data_x['verts'][0].cpu().numpy())
uv2_after = Pyx_after @ uv1
uv2_before = index_with_P(uv1, Pyx)

In [ ]:
#########################################
# X -> Y
#########################################

scene.geometry.clear()

# read /home/s94zalek_hpc/shape_matching/figures/texture.png with PIL
from PIL import Image

texture_img = Image.open('/home/s94zalek_hpc/shape_matching/figures/texture.png')

# create material
material=trimesh.visual.material.SimpleMaterial(
        image=texture_img,
        diffuse=[255, 255, 255, 255],
    )

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=data_x['verts'][0].cpu().numpy(), faces=data_x['faces'][0].cpu().numpy())

texture_visuals = trimesh.visual.texture.TextureVisuals(
    uv=uv1[:len(mesh1.vertices)],
    material=material
)

mesh1.visual = texture_visuals

scene.add_geometry(mesh1)


# correspondence we got from the functional map
mesh2_after = trimesh.Trimesh(vertices=data_y['verts'][0].cpu().numpy() + np.array([1, 0, 0]), faces=data_y['faces'][0].cpu().numpy())

texture_visuals_after = trimesh.visual.texture.TextureVisuals(
    uv=uv2_after[:len(mesh2_after.vertices)],
    material=material
)
mesh2_after.visual = texture_visuals_after

scene.add_geometry(mesh2_after)

scene.show()